Medir el tiempo de nodos en buffer de 2km al punto más cercano de López Mateos:

1. Hacer selección de los nodos que corresponden a López Mateos en un gdf manualmente (GIS?)
2. Descargar buffer 2km base de datos
3. graph_from_hippo para descargar edges_speed y nodes dentro del buffer
4. Museo_ciencias_ambientales>26cMCA .ipynb celda 34 aup_find_nearest
5. Museo_ciencias_ambientales>26cMCA .ipynb celd 35 aup.calculate_distance_nearest_poi
6. agrupar a hexres9